In [1]:
'''
notas

23 de março

acho que o calculo do return of investment para o caso de que não há nenhuma negociação tem que 
ser analisado com mais calma pois por exemplo supondo que na operação não haja nenhuma negociação
então o expo = 0, e o RoI vai ser profit/0. Se eu colocar o valor como 0, eu tenho problemas na 
otimização pois supondo que tu tenha uma operação A e uma operação B e a operação A possui RoI 
negativo, e a operação B não tem nenhuma negociação então o RoI vai ser dado como 0. Na otimização
a operação A vai ser pior que a operação B, pois -1 < 0, não sei se isso é a melhor solução pois
os parametros que pegam o 0 podem ser simplesmente parametros muito ruins pois não houve nenhuma 
negociação sequer. 

30 de marco

Ele ta comprando e vendendo no mesmo tempo.


~/Documents/github/Icarus/analisys/optmizer_analisys.py in __new__(self, name_x, name_y, df_treino, df_test, opt_param, n, pos_size, toTest, period, s_spread, l_spread, tp_s, tp_l, stop_s, stop_l)
    197         result_test = {}
    198         if(toTest):
--> 199             result_test = runbkt(df_test[name_x], df_test[name_y], name_x, name_y,
    200                                     opt_period, opt_shortspread, opt_longspread, opt_tpshort, opt_tplong, opt_stopshort, opt_stoplong, pos_size)
    201

KeyError: 'SQIA3'


~/Documents/github/Icarus/analisys/optmizer_analisys.py in __new__(self, name_x, name_y, df_treino, df_test, opt_param, n, pos_size, toTest, period, s_spread, l_spread, tp_s, tp_l, stop_s, stop_l)
    197         result_test = {}
    198         if(toTest):
--> 199             result_test = runbkt(df_test[name_x], df_test[name_y], name_x, name_y,
    200                                     opt_period, opt_shortspread, opt_longspread, opt_tpshort, opt_tplong, opt_stopshort, opt_stoplong, pos_size)
    201

KeyError: 'LOGG3'

stormer falou sobre grafico diario de 15 minutos
'''



"\nnotas\n\n23 de março\n\nacho que o calculo do return of investment para o caso de que não há nenhuma negociação tem que \nser analisado com mais calma pois por exemplo supondo que na operação não haja nenhuma negociação\nentão o expo = 0, e o RoI vai ser profit/0. Se eu colocar o valor como 0, eu tenho problemas na \notimização pois supondo que tu tenha uma operação A e uma operação B e a operação A possui RoI \nnegativo, e a operação B não tem nenhuma negociação então o RoI vai ser dado como 0. Na otimização\na operação A vai ser pior que a operação B, pois -1 < 0, não sei se isso é a melhor solução pois\nos parametros que pegam o 0 podem ser simplesmente parametros muito ruins pois não houve nenhuma \nnegociação sequer. \n\n30 de marco\n\nEle ta comprando e vendendo no mesmo tempo.\n\n\n~/Documents/github/Icarus/analisys/optmizer_analisys.py in __new__(self, name_x, name_y, df_treino, df_test, opt_param, n, pos_size, toTest, period, s_spread, l_spread, tp_s, tp_l, stop_s, stop_l)\

In [2]:
import init

from analisys.bkt_analisys import *
from analisys.data_analisys import *
from analisys.graph_analisys import *
from analisys.optmizer_analisys import *


import pandas as pd
import seaborn as sns
sns.set_theme()

import random
import os

from backend.resampler import *
from backend.mt5_tools import *

from statsmodels.tsa.stattools import adfuller
from analisys.graph_analisys import moving_avg

from secrets import token_hex as token


import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.io as pio
pio.orca.config.use_xvfb = True


In [3]:
#definindo parametros

start_date = '2021.01.01'
end_date = '2021.01.30'


param = 'profit'
csv_l = 'symbols_ibovf.csv'
note=''

dataframe = 10 # timeframe = 5 minutos
sample_size = 5 # quantidade de pares sendo negociados simultaneamente
n_opt = 45 # parametro do optmizer
lags = 25 # parametro do 'teste de cointegracao'
p_size = 300 # tamanho da posicao

d_train = 4 # numero de dias sendo 'treinados'.
d_test = 1 # numero de dias sendo 'testados'.

stop_n = [5,50]
stop_r = [-30, -10]
period = [5,100]
s_spread = [1.0, 6.0]
l_spread = [-6.0, -1.0]
tp_s =[-3.5, 3.5]
tp_l =[-3.5, 3.5]
tp_r =[80, 200]

In [ ]:
## cria a lista do espaço para a função de otimização

space_param = [(period[0], period[1]),
        (s_spread[0], s_spread[1]),
        (l_spread[0], l_spread[1]),
        (tp_s[0], tp_s[1]),
        (tp_l[0], tp_l[1]),
        (tp_r[0], tp_r[1]),
        (stop_n[0], stop_n[1]),
        (stop_r[0], stop_r[1])]



## pega os dados da ibov para ter uma lista de dias uteis no ano
ibov = mt5_singlehistoricaldata('IBOV', 16408, start_date, end_date)
ibov_date = list(ibov['date'])
ibov_date.reverse()



## cria a lista contendo tuplas com as datas para o inicio do treino, fim do treino, inicio do teste e fim do teste
date_iteration = []
for d in range(len(ibov_date))[:-(d_train+d_test)]:

    train_startdate = ibov_date[d].split(' ')[0]
    train_enddate = ibov_date[d+d_train].split(' ')[0]
    test_startdate = ibov_date[d+d_train].split(' ')[0]
    test_enddate = ibov_date[d+d_test+d_train].split(' ')[0]

    date_iteration.append(tuple([train_startdate, train_enddate,
                                test_startdate, test_enddate]))


real_startdate = date_iteration[0][2]
real_enddate = date_iteration[-1][-2]

df_loopdata = pd.DataFrame({'date0':[],
                            'date1':[],
                            'x':[],
                            'y':[],
                            'n':[],
                            'Profit':[],
                            'Roi':[],
                            'PPP':[],
                            'SR':[],
                            'PL':[],
                            'std':[],
                            'VaR90':[],
                            'VaR95':[],
                            'VaR99':[],
                            'info':[]})


df_testdata = pd.DataFrame({'date':[],
                            'RoI':[],
                            'PPP':[],
                            'EXPO':[],
                            'n':[]})


#loop de execução do programa

loop_date0_list = []
loop_date1_list = []
loop_return_list = []
loop_expo_list = []
loop_n_list= [] 
loop_ppp_list = []
loop_roi_list=[]


d_count = 1
code = token(2)

for d in date_iteration:

    print('{}/{}'.format(d_count, len(date_iteration)))
    train_startdate, train_enddate, test_startdate, test_enddate = d

    #pega os dados
    
    hist = mt5_historicaldata(dataframe, train_startdate, train_enddate, csv_list=csv_l)
    hist_test = mt5_historicaldata(dataframe, test_startdate, test_enddate, csv_list=csv_l)

    hist_keys = list(hist.keys())
    histtest_keys = list(hist_test.keys())

    if(len(histtest_keys) == 0 or len(hist_keys) == 0):

        df_testdata = df_testdata.append({'date':test_startdate,
                                        'RoI':0, 'PPP':0, 'EXPO':0, 'n':0}, ignore_index=True)

        d_count = d_count+1
        continue;


    for i in hist_keys:
        if(hist[i].empty):
            hist.pop(i)
            
    for i in histtest_keys:
        if(hist_test[i].empty):
            hist_test.pop(i)

    if(not (len(hist) == len(hist_test))):

        df_testdata = df_testdata.append({'date':test_startdate,
                                        'RoI':0, 'PPP':0, 'EXPO':0, 'n':0}, ignore_index=True)

        d_count = d_count+1
        continue


    #verifica cointegracao
    co = h_coint(hist, dataframe, n_lags=lags)
    co = pd.DataFrame(co)
    co = co.sort_values(by=['pvalue'])
    co = co.reset_index(drop=True)
    
    co
    
    breakpoint()
    new_co = pd.DataFrame()
    try:
        new_co = new_co.append(co.iloc()[0])
    except:
        df_testdata = df_testdata.append({'date':test_startdate,
                                        'RoI':0, 'PPP':0, 'EXPO':0, 'n':0}, ignore_index=True)
        
        d_count = d_count+1
        continue

    co_len = [i for i in range(len(co))]


    for j in (co_len):

        x,y = list(co['x'])[j], list(co['y'])[j]

        if((x in list(new_co['x'])) or (x in list(new_co['y'])) or 
            (y in list(new_co['x'])) or (y in list(new_co['y']))):
            3>2
        else:
            new_co = new_co.append(co.iloc()[j])


    new_co = new_co.reset_index(drop=True)
    co = pd.DataFrame(new_co)


    if(len(co) == 0):

        df_testdata = df_testdata.append({'date':test_startdate,
                                        'RoI':0,
                                        'PPP':0,
                                        'EXPO':0,
                                        'n':0}, ignore_index=True)

        d_count = d_count+1
        continue

    #seleciona pares aleatoriamente
    if(len(co) < sample_size):
        res = random.sample(range(len(co)), len(co))   
    else:
        res = random.sample(range(len(co)), sample_size)


    sample = pd.DataFrame(co.iloc()[res])
    breakpoint()
    sample_x = np.array(sample['x'])
    sample_y = np.array(sample['y'])


    #loop de execução dos backtests

    date0_list = []
    date1_list = []
    n_list = []
    profit_list = []
    expo_list = []
    roi_list = []
    ppp_list = []
    sr_list = []
    pl_list = []
    std_list = []
    var90_list =[]
    var95_list = []
    var99_list = []
    info_list = []


    bkt_counter = 1
    dcode = token(2)
    for i in range(len(sample_x)):

        x,y = sample_x[i], sample_y[i]


        result_opttrain = forestbkt(x, y, hist, hist_test, n=n_opt, opt_param=param, pos_size=p_size, space=space_param)



        bkt_date0 = result_opttrain['test']['info']['date'][0]
        bkt_date1 = result_opttrain['test']['info']['date'][1]
        bkt_info = result_opttrain['test']['info']
        bkt_n = result_opttrain['test']['amount_positions']
        bkt_testreturns = result_opttrain['test']['returns']
        bkt_trainreturns = result_opttrain['treino']['returns']


        if(bkt_n == 0):

            date0_list.append(bkt_date0)
            date1_list.append(bkt_date1)
            n_list.append(0)
            profit_list.append(0)
            expo_list.append(0)
            roi_list.append(0)
            ppp_list.append(0)
            sr_list.append(0)
            pl_list.append(0)
            std_list.append(0)
            var90_list.append(0)
            var95_list.append(0)
            var99_list.append(0)
            info_list.append(str(bkt_info))

            df_loopdata = df_loopdata.append({'date0':bkt_date0,
                                            'date1':bkt_date1,
                                            'x':x,
                                            'y':y,
                                            'n':0,
                                            'test_returns':bkt_testreturns,
                                            'train_returns':bkt_trainreturns,
                                            'Profit':0,
                                            'Roi':0,
                                            'PPP':0,
                                            'SR':0,
                                            'PL':0,
                                            'std':0,
                                            'VaR90':0,
                                            'VaR95':0,
                                            'VaR99':0,
                                            'info':bkt_info}, ignore_index=True)
            #print(df_loopdata)

            bkt_counter = bkt_counter + 1
            continue


        bkt_profit = result_opttrain['test']['profit']
        bkt_expo = result_opttrain['test']['avg_expo']
        bkt_roi = result_opttrain['test']['roi']
        '''
        try:
            bkt_roi = bkt_profit/bkt_expo
        except:
            bkt_roi = 0
        ''' 

        bkt_ppp = result_opttrain['test']['profit_per_position']
        bkt_sr = result_opttrain['test']['sucess_ratio']
        bkt_pl = result_opttrain['test']['average_profit_loss']
        bkt_std = result_opttrain['test']['std']
        bkt_var90 = result_opttrain['test']['var90']
        bkt_var95 = result_opttrain['test']['var95']
        bkt_var99 = result_opttrain['test']['var99']

        date0_list.append(bkt_date0)
        date1_list.append(bkt_date1)
        n_list.append(bkt_n)
        profit_list.append(bkt_profit)
        expo_list.append(bkt_expo)
        roi_list.append(bkt_roi)
        ppp_list.append(bkt_ppp)
        sr_list.append(bkt_sr)
        pl_list.append(bkt_pl)
        std_list.append(bkt_std)
        var90_list.append(bkt_var90)
        var95_list.append(bkt_var95)
        var99_list.append(bkt_var99)
        info_list.append(str(bkt_info))

        loopdata = {'date0':bkt_date0,
                    'date1':bkt_date1,
                    'x':x,
                    'y':y,
                    'n':bkt_n,
                    'test_returns':bkt_testreturns,
                    'train_returns':bkt_trainreturns,
                    'Profit':bkt_profit,
                    'Roi':bkt_roi,
                    'PPP':bkt_ppp,
                    'SR':bkt_sr,
                    'PL':bkt_pl,
                    'std':bkt_std,
                    'VaR90':bkt_var90,
                    'VaR95':bkt_var95,
                    'VaR99':bkt_var99,
                    'info':bkt_info}

        df_loopdata = df_loopdata.append(loopdata, ignore_index=True)
        opt_period, opt_shortspread,opt_longspread, opt_tpshort,opt_tplong, opt_tpreturn, opt_stopnumber, opt_stopreturn = result_opttrain['parametros']

        printdata = '{} {}{}x{} Profit:{:.2f} Roi:{} n:{} \nPPP:{:.4f} SR:{:.4f} PL:{:.4f} VaR90:{:.4f} std :{:.4f} \n[perido:{} s_spread: {:.2f} l_spread:{:.2f}\n tp_s:{:.2f} tp_l:{:.2f} tp_r:{:.2f}stop_n:{} stop_r:{:.2f}]\n\n\n'.format(loopdata['info']['date'][0],loopdata['info']['date'][1],
                                    loopdata['x'], loopdata['y'],
                                    loopdata['Profit'],bkt_roi,
                                    loopdata['n'],loopdata['PPP'],
                                    loopdata['SR'],loopdata['PL'],
                                    loopdata['VaR90'],loopdata['std'],
                                    opt_period, opt_shortspread, 
                                    opt_longspread, opt_tpshort, 
                                    opt_tplong, opt_tpreturn, 
                                    opt_stopnumber, opt_stopreturn)


        plname = '{}_{}_{}x{}_pl{}{}{}{}{}{}{}_{}x{}.jpg'.format(code, d_count, x, y,sample_size, n_opt, lags, param, p_size, d_train,d_test,start_date, end_date)

        profitplot_test = np.cumsum(result_opttrain['test']['returns'])
        profitrange_test = [i for i in range(len(profitplot_test))]

        profitplot_treino = np.cumsum(result_opttrain['treino']['returns'])
        profitrange_treino = [i for i in range(len(profitplot_treino))]

        fig, axes = plt.subplots(1,2)
        fig.suptitle(printdata, size='small')
        #axes[0].set_title(printdata)
        #axes[1].set_title('')



        sns.lineplot(ax=axes[0],x = profitrange_treino, y = profitplot_treino, marker='o')
        sns.lineplot(ax=axes[1],x = profitrange_test, y = profitplot_test, marker='o')

        #fig.set_title(printdata)

        fig.savefig(plname)
        fig.clf()

        #plt.plot(np.cumsum(result_opttrain['test']['returns']))
        print('--{}/{} {}\n{}\n'.format(bkt_counter, len(sample_x), printdata, result_opttrain['test']['info']))

        #print(df_loopdata)

        bkt_counter = bkt_counter + 1

    loop_date0 = bkt_date0
    loop_date0_list.append(bkt_date0)

    loop_date1 = bkt_date1
    loop_date1_list.append(bkt_date1)

    loop_return = sum(profit_list)
    if(loop_return == np.nan):
        loop_return = 0
    loop_return_list.append(loop_return)

    loop_expo = sum(expo_list)
    loop_expo_list.append(sum(expo_list)) 

    loop_n = sum(n_list)
    loop_n_list.append(sum(n_list)) 

    try:
        loop_ppp = loop_return/loop_n
        loop_ppp_list.append(loop_ppp)
    except:
        loop_ppp = 0
        loop_ppp_list.append(loop_ppp)

    try:
        loop_roi = loop_return/loop_expo
        loop_roi_list.append(loop_roi)
    except:
        loop_roi = 0
        loop_roi_list.append(loop_roi)

    df_testdata = df_testdata.append({'date':test_startdate,
                                    'return':loop_return,
                                    'RoI':loop_roi,
                                    'PPP':loop_ppp,
                                    'EXPO':loop_expo,
                                    'n':loop_n}, ignore_index=True)

    print(df_testdata)
    d_count = d_count + 1 

#adiciona informaçoes do 14bis e ibov
fourteenbis = np.cumsum(df_testdata['return']/np.mean(df_testdata['EXPO']))
ibov = mt5_singlehistoricaldata('IBOV', 16408, real_startdate, real_enddate)

df_ibov = pd.DataFrame(ibov).iloc[::-1]
df_ibov = df_ibov.reset_index().drop(columns=['index'])
ibov_percent = (np.array(df_ibov['close'])/df_ibov['close'][0] - 1)

df_testdata['IBOV'] = ibov_percent
df_testdata['14BIS'] = fourteenbis



tname='{}{}_t{}{}{}{}{}{}{}{}_{}x{}.xls'.format(space_param,code, dataframe, sample_size, n_opt, lags, param, p_size, d_train,d_test,start_date, end_date)
lname='{}{}_l{}{}{}{}{}{}{}{}_{}x{}.xls'.format(space_param,code, dataframe, sample_size, n_opt, lags, param, p_size, d_train,d_test,start_date, end_date)
pname='{}{}_p{}{}{}{}{}{}{}{}_{}x{}.png'.format(space_param,code, dataframe, sample_size, n_opt, lags, param, p_size, d_train,d_test,start_date, end_date)



df_testdata.to_excel(tname)
df_loopdata.to_excel(lname)

print(pname)


xs = ['{}'.format(i.split(' ')[0]) for i in ibov_date]
#print(xs)

pleg = 'df:{}|sample_size:{}|n_opt:{}|lags:{}|param:{}|\np_size:{}|d_train:{}|d_test:{}|csv_l:{}'.format(
    dataframe, sample_size, n_opt, lags, param, p_size, d_train, d_test, csv_l)

fig = make_subplots(rows=1, cols=1,specs= [[{'secondary_y':True}]])

fig.add_trace(go.Scatter(x=xs, y = df_testdata['14BIS']*100, name='14-bis', fill='tozeroy', mode='none'))
fig.add_trace(go.Scatter(x=xs, y = df_testdata['IBOV']*100, name='Ibovespa', fill='tozeroy', mode='none'))

fig.update_layout(xaxis_title=pleg, title={'text':str(space_param)})

fig['layout']['yaxis']['showgrid'] = False
img_bytes = fig.to_image(format="png", width=2000, height=2000, scale=5)
fig.write_image(pname)

1/14
['BBDC4', 'CIEL3', 'BBAS3', 'BPAC11', 'BBSE3', 'B3SA3', 'BBDC3', 'IRBR3', 'SANB11', 'ITSA4', 'BRML3', 'ITUB4', 'CYRE3', 'MULT3', 'SULA11', 'IGTA3']
> <ipython-input-4-8f53d5db9f3e>(123)<module>()
    121 
    122     breakpoint()
--> 123     new_co = pd.DataFrame()
    124     try:
    125         new_co = new_co.append(co.iloc()[0])

ipdb> continue
> <ipython-input-4-8f53d5db9f3e>(171)<module>()
    169     sample = pd.DataFrame(co.iloc()[res])
    170     breakpoint()
--> 171     sample_x = np.array(sample['x'])
    172     sample_y = np.array(sample['y'])
    173 

ipdb> sample
     pvalue      stdx      stdy      x      y
0  0.000648  5.000546  3.971769  IGTA3  CYRE3
ipdb> sample
     pvalue      stdx      stdy      x      y
0  0.000648  5.000546  3.971769  IGTA3  CYRE3
ipdb> sample
     pvalue      stdx      stdy      x      y
0  0.000648  5.000546  3.971769  IGTA3  CYRE3
ipdb> sample
     pvalue      stdx      stdy      x      y
0  0.000648  5.000546  3.971769  IGTA3  CYRE3
